In [3]:
from faker import Faker

from random import randint 
import pandas as pd 
import os
 
fake = Faker()
Faker.seed(42)
 
def input_data(x,product_types,customer_data):
   
    data = pd.DataFrame()
    Faker.seed(42)
#     Customer_id, order_id, prev_charms_count, prev_bracelet_count, prev_ring_count,
# this_order_charm_count, this_order_bracelet_count, this_order_ring_count
    for i in range(0, x):
        data.loc[i,'customer_id']  = fake.random_element(elements=(customer_data['customer_id']))
        data.loc [i,'order_id']= fake.uuid4()
        data.loc[i,'date']= fake.date_this_year()
        for j in range(0,len(product_types)):
            data.loc[i,'prev_' + product_types[j]+'_count'] = fake.random_int(min=0, max=10)
            data.loc[i,'this_' + product_types[j]+'_count'] = fake.random_int(min=0, max=5)
    return data.drop_duplicates()

def process_data(df,product_types,customer_data):
    for c in (range(len(customer_data))):
 
        dc=df.loc[df['customer_id'] == customer_data.loc[c,'customer_id']]
        dc_7=dc.sort_values(by=['customer_id', 'date'])
#         print("BEFORE")
#         display(dc_7)
#         Get the index_values which is the row number on the frame
        index_values= list(dc_7.index.values)
        for i in range(len(index_values)):
            if i != 0:
                for j in range(0,len(product_types)):
                    dc_7.loc[index_values[i],'prev_' + product_types[j]+'_count'] = dc_7.loc[index_values[i-1],'this_' + product_types[j]+'_count']
                    df.loc[index_values[i],'prev_' + product_types[j]+'_count'] = df.loc[index_values[i-1],'this_' + product_types[j]+'_count']
#         print("AFTER")
#         display(dc_7)     
    
    return df



########  MAIN   ####################################  
number_of_rows = 1000
product_types =['charm', 'bracelet', 'rings']
customer_data= pd.DataFrame()
Faker.seed(42)
for i in range(0,20):
    customer_data.loc[i,'customer_id']= "CUSTOMER_" + str(i)
# display(customer_data)
    

df=input_data(number_of_rows, product_types,customer_data)

df_processed=process_data(df,product_types,customer_data)

# display(df_processed)
os.makedirs('outputs', exist_ok=True)  

df_processed.to_csv('outputs/out.csv') 
